In [7]:
import sys
sys.path.append('../src')
import numpy as np


from utils import load_data, calculate_returns
from covariance import sample_cov

# Load data
prices = load_data(['AAPL', 'MSFT', 'GOOGL'], '2022-01-01', '2023-12-31')
returns = calculate_returns(prices)

# Calculate covariance
cov_mat = sample_cov(returns)
print(cov_mat.shape)  # Should be (3, 3) for 3 stocks

[*********************100%***********************]  3 of 3 completed

(3, 3)


# What's in this covariance matrix?

In [2]:
print("\nAnnualized Covariance Matrix:")
print(cov_mat)
print(f"\nMatrix shape: {cov_mat.shape}")


Annualized Covariance Matrix:
[[0.0845387  0.07143938 0.06618633]
 [0.07143938 0.12166652 0.07818557]
 [0.06618633 0.07818557 0.09440747]]

Matrix shape: (3, 3)


# Let's make it more readable with labels

In [3]:
import pandas as pd
tickers = ['AAPL', 'MSFT', 'GOOGL']
cov_df = pd.DataFrame(cov_mat, index=tickers, columns=tickers)
print("\nCovariance Matrix with labels:")
print(cov_df.round(4))


Covariance Matrix with labels:
         AAPL    MSFT   GOOGL
AAPL   0.0845  0.0714  0.0662
MSFT   0.0714  0.1217  0.0782
GOOGL  0.0662  0.0782  0.0944


# What do the diagonal values represent?

In [5]:
print("\nVariances (diagonal):")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {cov_mat[i,i]:.4f}")


Variances (diagonal):
AAPL: 0.0845
MSFT: 0.1217
GOOGL: 0.0944


# Convert variance to volatility (standard deviation)

In [8]:
print("\nAnnualized Volatility:")
for i, ticker in enumerate(tickers):
    vol = np.sqrt(cov_mat[i,i])
    print(f"{ticker}: {vol:.2%}")


Annualized Volatility:
AAPL: 29.08%
MSFT: 34.88%
GOOGL: 30.73%
